In [3]:
# ============================================
# CÉLULA 2: AUTO-CONFIGURAÇÃO HAIKU 4.5
# ============================================
import boto3
import json
import pandas as pd
from datetime import datetime
from xhtml2pdf import pisa
import os
from pathlib import Path
import pickle

print("📚 Bibliotecas importadas!\n")

# Configurar Bedrock Runtime
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name="us-east-2"
)

print("✅ Cliente AWS Bedrock configurado!")
print("🔍 Tentando configurar Claude Haiku 4.5...\n")

# ==============================================================
# ESTRATÉGIA: Tentar múltiplas formas de acessar Haiku 4.5
# ==============================================================

MODEL_ID = None
MODEL_NAME = None

# TENTATIVA 1: Usar Application Inference Profile (cross-region)
print("📌 Tentativa 1: Application Inference Profile...")
try:
    # Para cross-region inference, usamos um formato especial
    # que roteia automaticamente para a região com capacidade
    test_model_id = 'us.anthropic.claude-haiku-4-5-20251001-v1:0'
    
    # Testar com uma chamada mínima
    test_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10,
        "messages": [{"role": "user", "content": "test"}]
    }
    
    response = bedrock_client.invoke_model(
        body=json.dumps(test_config),
        modelId=test_model_id,
        accept="application/json",
        contentType="application/json"
    )
    
    MODEL_ID = test_model_id
    MODEL_NAME = "Claude Haiku 4.5"
    print("   ✅ SUCESSO! Claude Haiku 4.5 disponível!\n")
    
except Exception as e:
    print(f"   ❌ Não disponível: {str(e)[:100]}...\n")

# TENTATIVA 2: Inference Profile ARN direto
if not MODEL_ID:
    print("📌 Tentativa 2: ARN de Foundation Model...")
    try:
        test_model_id = 'arn:aws:bedrock:us-east-2::foundation-model/anthropic.claude-haiku-4-5-20251001-v1:0'
        
        test_config = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 10,
            "messages": [{"role": "user", "content": "test"}]
        }
        
        response = bedrock_client.invoke_model(
            body=json.dumps(test_config),
            modelId=test_model_id,
            accept="application/json",
            contentType="application/json"
        )
        
        MODEL_ID = test_model_id
        MODEL_NAME = "Claude Haiku 4.5"
        print("   ✅ SUCESSO via ARN!\n")
        
    except Exception as e:
        print(f"   ❌ Não disponível: {str(e)[:100]}...\n")

# TENTATIVA 3: Listar inference profiles disponíveis
if not MODEL_ID:
    print("📌 Tentativa 3: Buscando Inference Profiles...")
    try:
        bedrock_config = boto3.client('bedrock', region_name="us-east-2")
        response = bedrock_config.list_inference_profiles()
        
        # Procurar por Haiku 4.5
        for profile in response.get('inferenceProfileSummaries', []):
            profile_id = profile.get('inferenceProfileId', '').lower()
            profile_arn = profile.get('inferenceProfileArn', '')
            
            if 'haiku' in profile_id and '4' in profile_id:
                # Testar este profile
                try:
                    test_config = {
                        "anthropic_version": "bedrock-2023-05-31",
                        "max_tokens": 10,
                        "messages": [{"role": "user", "content": "test"}]
                    }
                    
                    response = bedrock_client.invoke_model(
                        body=json.dumps(test_config),
                        modelId=profile_arn,
                        accept="application/json",
                        contentType="application/json"
                    )
                    
                    MODEL_ID = profile_arn
                    MODEL_NAME = "Claude Haiku 4.5"
                    print(f"   ✅ SUCESSO! Profile encontrado: {profile_id}\n")
                    break
                    
                except:
                    continue
        
        if not MODEL_ID:
            print("   ⚠️  Nenhum profile Haiku 4.5 funcional encontrado\n")
            
    except Exception as e:
        print(f"   ❌ Erro ao listar profiles: {str(e)[:100]}...\n")

# FALLBACK: Claude 3.5 Haiku (SEMPRE FUNCIONA)
if not MODEL_ID:
    print("📌 Fallback: Usando Claude 3.5 Haiku")
    MODEL_ID = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
    MODEL_NAME = "Claude 3.5 Haiku"
    print("   ✅ Configurado!\n")

# ==============================================================
# RESUMO DA CONFIGURAÇÃO
# ==============================================================

print("="*60)
print(f"🤖 MODELO FINAL: {MODEL_NAME}")
print(f"📍 ID: {MODEL_ID}")
print("="*60)

if "4.5" in MODEL_NAME or "4-5" in MODEL_ID:
    print("\n🎉 ÓTIMA NOTÍCIA! Claude Haiku 4.5 está funcionando!")
    print("\n💰 CUSTOS ESTIMADOS:")
    print("   • Por questão: ~$0.0008")
    print("   • 220 questões: ~$0.18 USD")
    print("\n⭐ QUALIDADE:")
    print("   • Near-frontier performance")
    print("   • Melhor modelo custo-benefício disponível")
    print("   • Ideal para análises em escala")
else:
    print("\n💡 Usando Claude 3.5 Haiku (excelente opção!)")
    print("\n💰 CUSTOS ESTIMADOS:")
    print("   • Por questão: ~$0.003")
    print("   • 220 questões: ~$0.66 USD")
    print("\n⭐ QUALIDADE:")
    print("   • Excelente performance")
    print("   • Comprovado e estável")
    print("   • Com nosso prompt otimizado = Ótimos resultados")

print("\n" + "="*60)
print("✅ CONFIGURAÇÃO COMPLETA E TESTADA!")
print("📌 Próximo: Execute Célula 3")
print("="*60)

📚 Bibliotecas importadas!

✅ Cliente AWS Bedrock configurado!
🔍 Tentando configurar Claude Haiku 4.5...

📌 Tentativa 1: Application Inference Profile...
   ✅ SUCESSO! Claude Haiku 4.5 disponível!

🤖 MODELO FINAL: Claude Haiku 4.5
📍 ID: us.anthropic.claude-haiku-4-5-20251001-v1:0

🎉 ÓTIMA NOTÍCIA! Claude Haiku 4.5 está funcionando!

💰 CUSTOS ESTIMADOS:
   • Por questão: ~$0.0008
   • 220 questões: ~$0.18 USD

⭐ QUALIDADE:
   • Near-frontier performance
   • Melhor modelo custo-benefício disponível
   • Ideal para análises em escala

✅ CONFIGURAÇÃO COMPLETA E TESTADA!
📌 Próximo: Execute Célula 3


In [4]:
# ============================================
# CÉLULA 3: PROMPT OTIMIZADO PARA PORTUGUÊS
# ============================================

# PROMPT ESPECIALIZADO PARA ANÁLISE DE QUESTÕES DE PORTUGUÊS
# Adaptado para Sonnet 4.5 - Análise profunda e pedagógica

PROMPT_PORTUGUES_OTIMIZADO = """Você é um Professor Especialista em Língua Portuguesa com 15 anos de experiência analisando questões de concursos públicos.

MISSÃO: Criar análise pedagógica COMPLETA e PROFISSIONAL desta questão de Português.

╔══════════════════════════════════════════════════════════════╗
║  ESTRUTURA OBRIGATÓRIA DA ANÁLISE (SIGA RIGOROSAMENTE!)     ║
╚══════════════════════════════════════════════════════════════╝

1️⃣ IDENTIFICAÇÃO RÁPIDA
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🎯 Tema Central: [Ex: Concordância Verbal, Interpretação Textual, Figuras de Linguagem]
📚 Área: [GRAMÁTICA / INTERPRETAÇÃO / LITERATURA / REDAÇÃO OFICIAL]
📖 Subárea: [Ex: Sintaxe, Semântica, Morfologia, Compreensão, etc.]
🎓 Nível: [BÁSICO / INTERMEDIÁRIO / AVANÇADO]
🎪 Padrão da Banca: [Tipo de enunciado usado]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


2️⃣ CONCEITOS-CHAVE (Memorize!)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Liste 4-5 conceitos linguísticos essenciais:

✓ Conceito 1: [Nome] → [Definição técnica em 1 linha]
✓ Conceito 2: [Nome] → [Definição técnica em 1 linha]
✓ Conceito 3: [Nome] → [Definição técnica em 1 linha]
✓ Conceito 4: [Nome] → [Definição técnica em 1 linha]
✓ Conceito 5: [Nome] → [Definição técnica em 1 linha]

📝 REGRA GRAMATICAL APLICÁVEL (se for questão de gramática):
[Cite a regra específica da norma culta]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


3️⃣ ANÁLISE DE CADA ALTERNATIVA (CRÍTICO!)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Para CADA alternativa, estruture assim:

┌─────────────────────────────────────────────────────────────┐
│ 🔤 ALTERNATIVA A                                            │
└─────────────────────────────────────────────────────────────┘

📄 Conteúdo: [Reproduza ou resuma a alternativa]

Status: ❌ INCORRETA  (ou ✅ CORRETA)

📖 Fundamento Linguístico:
[Cite regra gramatical, teoria literária, ou técnica de interpretação]
[Ex: "Segundo a NBV (Nomenclatura Gramatical Brasileira)..."]
[Ex: "Conforme Bechara/Cunha & Cintra/Celso Pedro Luft..."]

💡 Análise Técnica Detalhada:
[Para GRAMÁTICA: Explique a estrutura sintática, morfológica ou semântica]
[Para INTERPRETAÇÃO: Analise o texto-base, identifique inferências]
[Para LITERATURA: Contextualize escola literária, autor, características]
[Identifique elementos-chave: palavras, expressões, estruturas]

🔍 Análise Específica:
- Classe gramatical: [Se aplicável]
- Função sintática: [Se aplicável]
- Relação semântica: [Se aplicável]
- Elementos textuais: [Se aplicável]

⚠️ Pegadinha (se houver):
[Explique a armadilha da banca: troca de sentido, confusão de termos, etc.]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

┌─────────────────────────────────────────────────────────────┐
│ 🔤 ALTERNATIVA B                                            │
└─────────────────────────────────────────────────────────────┘

[Repita estrutura completa]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

┌─────────────────────────────────────────────────────────────┐
│ 🔤 ALTERNATIVA C                                            │
└─────────────────────────────────────────────────────────────┘

[Repita estrutura completa]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

┌─────────────────────────────────────────────────────────────┐
│ 🔤 ALTERNATIVA D                                            │
└─────────────────────────────────────────────────────────────┘

[Repita estrutura completa]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

┌─────────────────────────────────────────────────────────────┐
│ 🔤 ALTERNATIVA E                                            │
└─────────────────────────────────────────────────────────────┘

[Repita estrutura completa]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


4️⃣ GABARITO DEFINITIVO
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ GABARITO: {gabarito}

🎯 Por que esta é a ÚNICA correta:
[Reforce o raciocínio linguístico em 2-3 linhas]
[Cite a regra ou princípio que fundamenta]

📝 Resumo em 1 frase:
[Síntese técnica da resposta correta]

🔑 Palavra-chave da questão:
[Identifique o termo ou expressão que resolve a questão]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


5️⃣ PEGADINHAS IDENTIFICADAS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Liste TODAS as pegadinhas desta questão:

⚠️ Pegadinha 1: [Ex: Troca de sentido sutil, paronímia]
⚠️ Pegadinha 2: [Ex: Estrutura sintática similar mas com erro]
⚠️ Pegadinha 3: [Ex: Interpretação literal vs. inferência]

🚫 ARMADILHAS LINGUÍSTICAS encontradas:
- Palavras confundíveis: [Liste pares como mal/mau, mas/mais]
- Estruturas ambíguas: [Identifique]
- Mudanças de sentido: [Destaque]
- Erros sutis: [Aponte]

📊 DISTRATORES (alternativas incorretas):
[Explique por que as incorretas parecem corretas]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


6️⃣ ESTRATÉGIA DE RESOLUÇÃO
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📚 COMO ESTUDAR:
- [Dica prática 1 - Ex: "Revise regras de concordância verbal"]
- [Dica prática 2 - Ex: "Pratique identificação de funções sintáticas"]
- [Dica prática 3 - Ex: "Leia textos do mesmo gênero"]

🔗 TEMAS RELACIONADOS PARA REVISAR:
- [Tema 1 - Ex: Regência verbal]
- [Tema 2 - Ex: Colocação pronominal]
- [Tema 3 - Ex: Tipologia textual]

📖 GRAMÁTICOS DE REFERÊNCIA:
[Cite Bechara, Cunha & Cintra, Celso Luft, ou outros conforme tema]

✍️ DICA DE PROVA (OURO!):
[Uma estratégia rápida para resolver questões similares]

⏱️ TEMPO IDEAL DE RESOLUÇÃO:
Iniciante: [X min] | Experiente: [Y min]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


7️⃣ ANÁLISE FINAL
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 Dificuldade: [BÁSICO/INTERMEDIÁRIO/AVANÇADO]
Justificativa: [Por quê?]

🎯 Competência Avaliada:
[ ] Conhecimento gramatical
[ ] Interpretação textual
[ ] Análise sintática
[ ] Conhecimento literário
[ ] Domínio semântico
[ ] Norma culta vs. coloquial

📈 Frequência em Concursos:
Este tema é: [Muito Cobrado / Médio / Raro]

🎓 TIPO DE QUESTÃO:
[ ] Decoreba (requer memorização)
[ ] Raciocínio (requer análise)
[ ] Interpretação (requer compreensão textual)
[ ] Mista

💡 DICA FINAL DE OURO:
[Uma dica valiosa para o concurseiro dominar este tema]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╔══════════════════════════════════════════════════════════════╗
║  REGRAS OBRIGATÓRIAS (NÃO PULE NADA!)                       ║
╚══════════════════════════════════════════════════════════════╝

✓ Analise TODAS as 5 alternativas com a estrutura completa
✓ Cite fundamentos linguísticos ESPECÍFICOS (gramáticos, regras)
✓ Identifique TODAS as pegadinhas e armadilhas
✓ Para GRAMÁTICA: cite a regra e exemplifique
✓ Para INTERPRETAÇÃO: relacione com o texto-base
✓ Para LITERATURA: contextualize movimento/autor
✓ Use linguagem TÉCNICA mas DIDÁTICA
✓ Seja DETALHADO (mínimo 3000 caracteres)
✓ Marque claramente ✅ CORRETA ou ❌ INCORRETA
✓ Mantenha a formatação com emojis e linhas
✓ Se houver texto-base, referencie trechos específicos

═══════════════════════════════════════════════════════════════

📊 DADOS DA QUESTÃO:

CONTEXTO:
- Banca: {banca}
- Ano: {ano}
- Órgão: {orgao}
- Prova: {prova}
- Disciplina: {disciplina}
- Assunto: {assunto}

ENUNCIADO:
{enunciado}

ALTERNATIVAS:
A) {alt_a}
B) {alt_b}
C) {alt_c}
D) {alt_d}
E) {alt_e}

GABARITO OFICIAL: {gabarito}

═══════════════════════════════════════════════════════════════

IMPORTANTE: Sua análise será avaliada por:
✓ Completude (analisou todas alternativas?)
✓ Profundidade (fundamentos linguísticos citados?)
✓ Clareza (estrutura organizada?)
✓ Precisão técnica (termos corretos da gramática?)
✓ Utilidade pedagógica (ensina de verdade?)

ATENÇÃO ESPECIAL:
- Se for questão de INTERPRETAÇÃO: analise o texto-base linha por linha
- Se for questão de GRAMÁTICA: cite a regra e dê exemplos
- Se for questão de LITERATURA: contextualize escola literária
- Se for questão de REDAÇÃO OFICIAL: cite manual (MRPR 3ª ed.)

Comece agora com "1️⃣ IDENTIFICAÇÃO RÁPIDA":"""

print("✅ Prompt ULTRA-OTIMIZADO para PORTUGUÊS configurado!")
print("\n📋 Características do prompt:")
print("   ✓ Adaptado para questões de Língua Portuguesa")
print("   ✓ Cobre: Gramática, Interpretação, Literatura, Redação")
print("   ✓ Estrutura visual clara com emojis e separadores")
print("   ✓ Instruções específicas para cada tipo de questão")
print("   ✓ Referências a gramáticos e norma culta")
print("   ✓ Análise de pegadinhas linguísticas")
print("   ✓ Validação de qualidade pedagógica")
print("\n" + "="*60)
print("📌 Próximo: Execute Célula 4")
print("="*60)

✅ Prompt ULTRA-OTIMIZADO para PORTUGUÊS configurado!

📋 Características do prompt:
   ✓ Adaptado para questões de Língua Portuguesa
   ✓ Cobre: Gramática, Interpretação, Literatura, Redação
   ✓ Estrutura visual clara com emojis e separadores
   ✓ Instruções específicas para cada tipo de questão
   ✓ Referências a gramáticos e norma culta
   ✓ Análise de pegadinhas linguísticas
   ✓ Validação de qualidade pedagógica

📌 Próximo: Execute Célula 4


In [5]:
# ============================================
# CÉLULA 4: ANÁLISE OTIMIZADA PARA PORTUGUÊS
# ============================================

# Criar pasta de cache
CACHE_DIR = Path('cache_analises_portugues')
CACHE_DIR.mkdir(exist_ok=True)

def validar_qualidade_analise(analise, questao_dict):
    """Valida qualidade da análise de questões de Português"""
    score = 0
    
    # 1. Tamanho adequado
    tamanho = len(analise)
    if tamanho >= 3500:
        score += 20
    elif tamanho >= 2500:
        score += 15
    elif tamanho >= 1800:
        score += 10
    
    # 2. Análise de TODAS alternativas
    alternativas_presentes = sum([
        'ALTERNATIVA A' in analise.upper(),
        'ALTERNATIVA B' in analise.upper(),
        'ALTERNATIVA C' in analise.upper(),
        'ALTERNATIVA D' in analise.upper(),
        'ALTERNATIVA E' in analise.upper()
    ])
    score += alternativas_presentes * 12
    
    # 3. Marcação clara de corretas/incorretas
    tem_marcacoes = analise.count('✅') + analise.count('❌')
    if tem_marcacoes >= 5:
        score += 10
    elif tem_marcacoes >= 3:
        score += 5
    
    # 4. Fundamentos linguísticos (adaptado para Português)
    termos_linguisticos = [
        'regra' in analise.lower() or 'norma' in analise.lower(),
        'gramática' in analise.lower() or 'gramatical' in analise.lower(),
        'semântica' in analise.lower() or 'sintaxe' in analise.lower(),
        'bechara' in analise.lower() or 'cunha' in analise.lower() or 'luft' in analise.lower(),
        'classe' in analise.lower() or 'função' in analise.lower(),
        'interpretação' in analise.lower() or 'compreensão' in analise.lower()
    ]
    score += sum(termos_linguisticos) * 2
    
    # 5. Estrutura organizada
    tem_estrutura = all([
        'IDENTIFICAÇÃO' in analise.upper() or '1️⃣' in analise,
        'CONCEITOS' in analise.upper() or '2️⃣' in analise,
        'GABARITO' in analise.upper()
    ])
    if tem_estrutura:
        score += 10
    
    # 6. Análise de pegadinhas linguísticas
    if 'pegadinha' in analise.lower() or '⚠️' in analise:
        score += 8
    
    # 7. Análise técnica específica de Português
    tem_analise_tecnica = any([
        'classe gramatical' in analise.lower(),
        'função sintática' in analise.lower(),
        'concordância' in analise.lower(),
        'regência' in analise.lower(),
        'colocação' in analise.lower(),
        'figura de linguagem' in analise.lower()
    ])
    if tem_analise_tecnica:
        score += 5
    
    return min(score, 100)


def analisar_questao_com_ia_otimizada(questao_dict, max_tentativas=2):
    """
    Análise otimizada de questões de Português com retry inteligente
    """
    
    questao_id = questao_dict.get('id_questao', 'unknown')
    cache_file = CACHE_DIR / f"{questao_id}.pkl"
    
    # Verificar cache
    if cache_file.exists():
        print(f"   💾 Cache encontrado!")
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    
    # Gerar análise
    for tentativa in range(1, max_tentativas + 1):
        try:
            if tentativa > 1:
                print(f"   🔄 Tentativa {tentativa}/{max_tentativas}...")
            
            # Montar prompt
            prompt_final = PROMPT_PORTUGUES_OTIMIZADO.format(
                banca=questao_dict.get('banca', 'N/A'),
                ano=questao_dict.get('ano', 'N/A'),
                orgao=questao_dict.get('orgao', 'N/A'),
                prova=questao_dict.get('prova', 'N/A'),
                disciplina=questao_dict.get('disciplina', 'N/A'),
                assunto=questao_dict.get('assunto', 'N/A'),
                enunciado=questao_dict.get('enunciado', 'N/A'),
                alt_a=questao_dict.get('alt_a', 'N/A'),
                alt_b=questao_dict.get('alt_b', 'N/A'),
                alt_c=questao_dict.get('alt_c', 'N/A'),
                alt_d=questao_dict.get('alt_d', 'N/A'),
                alt_e=questao_dict.get('alt_e', 'N/A'),
                gabarito=questao_dict.get('gabarito', 'N/A')
            )
            
            # ✅ CONFIGURAÇÃO CORRIGIDA (SEM top_p)
            config = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 5000,
                "temperature": 0.3,  # ✅ Só temperature
                "system": """Você é um Professor PhD em Língua Portuguesa, especialista em análise pedagógica de questões de concursos.

SUAS CARACTERÍSTICAS:
- 15 anos de experiência em concursos públicos
- Taxa de aprovação dos alunos: 87%
- Especialidade: Identificar pegadinhas linguísticas
- Domínio completo: Gramática, Interpretação, Literatura, Redação
- Método: Análises completas, técnicas e didáticas

SEUS PRINCÍPIOS:
1. SEMPRE analisar TODAS as 5 alternativas
2. SEMPRE citar fundamentos linguísticos específicos (gramáticos, regras NBG)
3. SEMPRE identificar pegadinhas e armadilhas
4. SEMPRE manter estrutura clara com emojis
5. SEMPRE ser técnico mas didático
6. Para GRAMÁTICA: citar regras e exemplificar
7. Para INTERPRETAÇÃO: referenciar o texto-base
8. Para LITERATURA: contextualizar escola/autor

Você NÃO aceita análises superficiais. Cada resposta deve ENSINAR profundamente.""",
                "messages": [
                    {
                        "role": "user",
                        "content": prompt_final
                    }
                ]
            }
            
            # Invocar Bedrock
            response = bedrock_client.invoke_model(
                body=json.dumps(config),
                modelId=MODEL_ID,
                accept="application/json",
                contentType="application/json"
            )
            
            # Extrair resposta
            resposta = json.loads(response['body'].read().decode('utf-8'))
            analise = resposta.get('content', [{}])[0].get('text', 'Erro')
            
            # Validar qualidade
            score = validar_qualidade_analise(analise, questao_dict)
            print(f"   📊 Score de qualidade: {score}/100")
            
            # Se qualidade boa, salvar e retornar
            if score >= 65:
                print(f"   ✅ Análise aprovada!")
                
                # Salvar no cache
                with open(cache_file, 'wb') as f:
                    pickle.dump(analise, f)
                
                return analise
            
            # Se qualidade baixa e ainda tem tentativas, ajustar
            elif tentativa < max_tentativas:
                print(f"   ⚠️  Qualidade abaixo do ideal, ajustando...")
                # Na próxima tentativa, aumentar temperature
                config["temperature"] = 0.4
                
        except Exception as e:
            print(f"   ❌ Erro: {str(e)[:100]}")
            if tentativa == max_tentativas:
                return f"❌ Erro após {max_tentativas} tentativas: {str(e)}"
            continue
    
    # Retornar última tentativa mesmo se não ideal
    with open(cache_file, 'wb') as f:
        pickle.dump(analise, f)
    return analise


print("✅ Sistema de análise para PORTUGUÊS criado!")
print("\n🔧 Configuração:")
print("   ✅ Modelo: Claude Sonnet 4.5")
print("   ✅ Temperature: 0.3 (sem top_p)")
print("   ✅ Max tokens: 5000")
print("\n📊 Funcionalidades:")
print("   ✓ Cache inteligente em: cache_analises_portugues/")
print("   ✓ Retry automático (até 2 tentativas)")
print("   ✓ Validação de qualidade linguística")
print("   ✓ Score mínimo: 65/100")
print("\n🎯 Critérios de validação:")
print("   ✓ Análise de todas as 5 alternativas")
print("   ✓ Fundamentos linguísticos citados")
print("   ✓ Estrutura organizada com emojis")
print("   ✓ Identificação de pegadinhas")
print("   ✓ Análise técnica (gramática/interpretação)")
print("\n" + "="*60)
print("✅ PRONTO PARA ANALISAR QUESTÕES DE PORTUGUÊS!")
print("📌 Próximo: Execute Célula 5 (carregar questões)")
print("="*60)

✅ Sistema de análise para PORTUGUÊS criado!

🔧 Configuração:
   ✅ Modelo: Claude Sonnet 4.5
   ✅ Temperature: 0.3 (sem top_p)
   ✅ Max tokens: 5000

📊 Funcionalidades:
   ✓ Cache inteligente em: cache_analises_portugues/
   ✓ Retry automático (até 2 tentativas)
   ✓ Validação de qualidade linguística
   ✓ Score mínimo: 65/100

🎯 Critérios de validação:
   ✓ Análise de todas as 5 alternativas
   ✓ Fundamentos linguísticos citados
   ✓ Estrutura organizada com emojis
   ✓ Identificação de pegadinhas
   ✓ Análise técnica (gramática/interpretação)

✅ PRONTO PARA ANALISAR QUESTÕES DE PORTUGUÊS!
📌 Próximo: Execute Célula 5 (carregar questões)


In [6]:
# ============================================
# CÉLULA 5: FUNÇÃO DE GERAÇÃO DE PDF
# ============================================

def gerar_pdf_questao(questao_dict, analise_ia, output_path):
    """
    Gera PDF formatado profissionalmente para questões de Português
    """
    
    # Função para escapar HTML
    def escape_html(text):
        if pd.isna(text):
            return 'N/A'
        return (str(text)
                .replace('&', '&amp;')
                .replace('<', '&lt;')
                .replace('>', '&gt;')
                .replace('"', '&quot;')
                .replace("'", '&#39;'))
    
    # Converter análise para HTML
    analise_html = escape_html(analise_ia)
    analise_html = (analise_html
                   .replace('\n\n', '</p><p>')
                   .replace('\n', '<br/>')
                   .replace('**', '<b>')
                   .replace('**', '</b>'))
    
    # Gerar HTML das alternativas
    def gerar_html_alternativas(q):
        gabarito = str(q.get('gabarito', '')).strip().upper()
        alternativas_html = ""
        
        for letra in ['A', 'B', 'C', 'D', 'E']:
            texto = escape_html(q.get(f'alt_{letra.lower()}', 'N/A'))
            eh_correta = (letra == gabarito)
            
            if eh_correta:
                estilo = 'background: linear-gradient(135deg, #d4edda 0%, #c3e6cb 100%); border-left: 5px solid #28a745; padding: 15px; margin: 12px 0; border-radius: 6px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);'
                label_estilo = 'background-color: #28a745; color: white; padding: 6px 14px; border-radius: 5px; font-weight: bold; margin-right: 12px; font-size: 13pt;'
                simbolo = '✅'
            else:
                estilo = 'background-color: #f8f9fa; border-left: 4px solid #dee2e6; padding: 15px; margin: 12px 0; border-radius: 6px;'
                label_estilo = 'background-color: #6c757d; color: white; padding: 6px 14px; border-radius: 5px; font-weight: bold; margin-right: 12px; font-size: 13pt;'
                simbolo = '⬜'
            
            alternativas_html += f'''
            <div style="{estilo}">
                <span style="{label_estilo}">{letra}</span>
                <span style="font-weight: bold; font-size: 12pt;">{simbolo}</span>
                <span style="font-size: 11pt;">{texto}</span>
            </div>
            '''
        
        return alternativas_html
    
    # Template HTML
    html_template = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8"/>
        <style>
            @page {{ size: a4 portrait; margin: 1.8cm; }}
            body {{ font-family: Arial; font-size: 11pt; line-height: 1.7; color: #2c3e50; }}
            .header {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 28px; text-align: center; margin-bottom: 25px; border-radius: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); }}
            .header h1 {{ margin: 0 0 12px 0; font-size: 26pt; font-weight: bold; }}
            .metadata {{ font-size: 11pt; opacity: 0.95; line-height: 1.6; }}
            .metadata-badge {{ background-color: rgba(255,255,255,0.2); padding: 4px 12px; border-radius: 15px; margin: 3px; display: inline-block; }}
            .section {{ margin-bottom: 22px; }}
            .section-title {{ background: linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%); padding: 14px 20px; border-left: 6px solid #667eea; font-size: 15pt; font-weight: bold; margin-bottom: 16px; color: #2c3e50; border-radius: 5px; }}
            .info-grid {{ display: table; width: 100%; margin: 15px 0; }}
            .info-row {{ display: table-row; }}
            .info-label {{ display: table-cell; background-color: #f8f9fa; font-weight: bold; padding: 12px; border: 1px solid #dee2e6; width: 30%; }}
            .info-value {{ display: table-cell; padding: 12px; border: 1px solid #dee2e6; background-color: white; }}
            .enunciado {{ background: linear-gradient(135deg, #fffbea 0%, #fff8dc 100%); border: 3px solid #ffd93d; padding: 20px; margin: 18px 0; font-size: 12pt; line-height: 1.8; border-radius: 8px; }}
            .gabarito-box {{ background: linear-gradient(135deg, #28a745 0%, #20c997 100%); color: white; padding: 25px; text-align: center; font-size: 20pt; font-weight: bold; margin: 25px 0; border-radius: 10px; }}
            .analise-ia {{ background: linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%); border: 3px solid #2196f3; padding: 20px; margin: 20px 0; border-radius: 8px; line-height: 1.9; }}
            .footer {{ margin-top: 40px; padding-top: 20px; border-top: 3px solid #dee2e6; text-align: center; font-size: 9pt; color: #6c757d; }}
            .id-badge {{ background-color: #6c757d; color: white; padding: 5px 12px; border-radius: 5px; font-size: 9pt; font-weight: bold; display: inline-block; margin-top: 10px; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1>📋 ANÁLISE PEDAGÓGICA - PORTUGUÊS</h1>
            <div class="metadata">
                <span class="metadata-badge">🏛️ {escape_html(questao_dict.get('banca', 'N/A'))}</span>
                <span class="metadata-badge">📅 {escape_html(questao_dict.get('ano', 'N/A'))}</span>
                <span class="metadata-badge">🏢 {escape_html(questao_dict.get('orgao', 'N/A'))}</span>
                <br/><span style="font-size: 10pt;">{escape_html(questao_dict.get('prova', 'N/A'))}</span>
            </div>
        </div>
        
        <div class="section">
            <div class="section-title">📊 IDENTIFICAÇÃO</div>
            <div class="info-grid">
                <div class="info-row"><div class="info-label">📚 Disciplina</div><div class="info-value">{escape_html(questao_dict.get('disciplina', 'N/A'))}</div></div>
                <div class="info-row"><div class="info-label">📖 Assunto</div><div class="info-value">{escape_html(questao_dict.get('assunto', 'N/A'))}</div></div>
                <div class="info-row"><div class="info-label">🏛️ Banca</div><div class="info-value">{escape_html(questao_dict.get('banca', 'N/A'))}</div></div>
                <div class="info-row"><div class="info-label">📅 Ano</div><div class="info-value">{escape_html(questao_dict.get('ano', 'N/A'))}</div></div>
                <div class="info-row"><div class="info-label">🏢 Órgão</div><div class="info-value">{escape_html(questao_dict.get('orgao', 'N/A'))}</div></div>
                <div class="info-row"><div class="info-label">✅ Gabarito</div><div class="info-value"><b style="color: #28a745; font-size: 14pt;">{escape_html(questao_dict.get('gabarito', 'N/A'))}</b></div></div>
            </div>
            <div class="id-badge">ID: {escape_html(questao_dict.get('id_questao', 'N/A'))}</div>
        </div>
        
        <div class="section">
            <div class="section-title">📝 ENUNCIADO</div>
            <div class="enunciado">{escape_html(questao_dict.get('enunciado', 'N/A'))}</div>
        </div>
        
        <div class="section">
            <div class="section-title">🔤 ALTERNATIVAS</div>
            {gerar_html_alternativas(questao_dict)}
        </div>
        
        <div class="gabarito-box">✅ GABARITO: ALTERNATIVA {escape_html(questao_dict.get('gabarito', 'N/A'))}</div>
        
        <div class="section">
            <div class="section-title">🤖 ANÁLISE PEDAGÓGICA COMPLETA</div>
            <div class="analise-ia"><p>{analise_html}</p></div>
        </div>
        
        <div class="footer">
            📚 Análise Linguística Profissional<br/>
            Gerada via <b>AWS Bedrock - Claude Sonnet 4.5</b><br/>
            Sistema de Análise Pedagógica para Português v2.0<br/>
            {datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}
        </div>
    </body>
    </html>
    '''
    
    # Gerar PDF
    try:
        with open(output_path, "wb") as pdf_file:
            pisa_status = pisa.CreatePDF(html_template, dest=pdf_file)
        
        if not pisa_status.err:
            print(f"   ✅ PDF salvo: {output_path}")
            return True
        else:
            print(f"   ❌ Erro ao gerar PDF")
            return False
    except Exception as e:
        print(f"   ❌ Erro ao gerar PDF: {str(e)}")
        raise

print("✅ Função de geração de PDF para PORTUGUÊS criada!")
print("\n📄 Características do PDF:")
print("   ✓ Layout profissional com gradientes")
print("   ✓ Gabarito destacado em verde")
print("   ✓ Alternativas formatadas com cores")
print("   ✓ Análise pedagógica completa")
print("   ✓ Identificação clara do modelo (Sonnet 4.5)")
print("   ✓ Cabeçalho personalizado para Português")
print("\n" + "="*60)
print("📌 Próximo: Execute Célula 6 (carregar questões)")
print("="*60)

✅ Função de geração de PDF para PORTUGUÊS criada!

📄 Características do PDF:
   ✓ Layout profissional com gradientes
   ✓ Gabarito destacado em verde
   ✓ Alternativas formatadas com cores
   ✓ Análise pedagógica completa
   ✓ Identificação clara do modelo (Sonnet 4.5)
   ✓ Cabeçalho personalizado para Português

📌 Próximo: Execute Célula 6 (carregar questões)


In [7]:
# ============================================
# CÉLULA 6: CARREGAR DATASET DE PORTUGUÊS
# ============================================

# 📝 INSTRUÇÕES: Atualize o caminho para seu arquivo de questões de Português
CAMINHO_ARQUIVO = 'C:\welligton-projects-IA\RPA-PROJETO-1\questoes_PORTUGUES_TODAS_FGV_TJ.xlsx'

print(f"📂 Carregando dataset de Português...")
print("⏳ Aguarde...\n")

try:
    df = pd.read_excel(CAMINHO_ARQUIVO)
    
    print(f"✅ Dataset carregado!")
    print(f"📊 Total: {len(df)} questões\n")
    
    print("📈 ESTATÍSTICAS:")
    print(f"   🏛️  Banca: {df['Banca'].unique()[0] if 'Banca' in df.columns else 'N/A'}")
    
    if 'Ano' in df.columns:
        anos_disponiveis = sorted(df['Ano'].dropna().unique())
        print(f"   📅 Anos: {anos_disponiveis}")
    
    if 'Orgao' in df.columns:
        print(f"   🏢 Órgãos: {df['Orgao'].nunique()} diferentes")
    
    if 'Assunto' in df.columns:
        print(f"   📚 Assuntos: {df['Assunto'].nunique()} temas diferentes")
        print(f"\n📖 Principais assuntos:")
        top_assuntos = df['Assunto'].value_counts().head(5)
        for assunto, count in top_assuntos.items():
            print(f"      • {assunto}: {count} questões")
    
    print("\n📊 Distribuição de Gabaritos:")
    if 'Resposta_Correta' in df.columns:
        for gab, count in df['Resposta_Correta'].value_counts().sort_index().items():
            porc = (count / len(df)) * 100
            barra = '█' * int(porc / 2)
            print(f"   Alt {gab}: {count:3d} ({porc:5.1f}%) {barra}")
    
    print("\n🔍 Colunas encontradas no dataset:")
    print(f"   {', '.join(df.columns.tolist())}")
    
    print("\n" + "="*60)
    print("✅ DATASET PRONTO PARA ANÁLISE!")
    print("📌 Próximo: Execute Célula 7 para TESTAR")
    print("="*60)
    
except FileNotFoundError:
    print(f"❌ ERRO: Arquivo não encontrado!")
    print(f"\n📁 Caminho atual: {CAMINHO_ARQUIVO}")
    print("\n💡 SOLUÇÃO:")
    print("   1. Verifique se o arquivo existe")
    print("   2. Copie o caminho correto do arquivo")
    print("   3. Atualize a variável CAMINHO_ARQUIVO acima")
    print("   4. Use barras duplas (\\\\) no Windows")
    print("\n📝 Exemplo:")
    print("   CAMINHO_ARQUIVO = 'C:\\\\Users\\\\Seu Nome\\\\questoes_portugues.xlsx'")
    
except Exception as e:
    print(f"❌ ERRO: {str(e)}")
    print(f"\n🔍 Detalhes: {type(e).__name__}")

<>:6: SyntaxWarning: invalid escape sequence '\w'
<>:6: SyntaxWarning: invalid escape sequence '\w'
C:\Users\welligton.costa.s\AppData\Local\Temp\ipykernel_10228\3461922011.py:6: SyntaxWarning: invalid escape sequence '\w'
  CAMINHO_ARQUIVO = 'C:\welligton-projects-IA\RPA-PROJETO-1\questoes_PORTUGUES_TODAS_FGV_TJ.xlsx'


📂 Carregando dataset de Português...
⏳ Aguarde...

✅ Dataset carregado!
📊 Total: 180 questões

📈 ESTATÍSTICAS:
   🏛️  Banca: FGV
   📅 Anos: [np.int64(2008), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
   🏢 Órgãos: 18 diferentes
   📚 Assuntos: 6 temas diferentes

📖 Principais assuntos:
      • Morfologia,: 88 questões
      • Morfologia - Verbos,: 53 questões
      • Ortografia,: 23 questões
      • Interpretação de Textos,: 13 questões
      • Sintaxe,: 2 questões

📊 Distribuição de Gabaritos:
   Alt A:  41 ( 22.8%) ███████████
   Alt B:  37 ( 20.6%) ██████████
   Alt C:  35 ( 19.4%) █████████
   Alt D:  26 ( 14.4%) ███████
   Alt E:  41 ( 22.8%) ███████████

🔍 Colunas encontradas no dataset:
   ID_Questao, Indice_Pagina, Ano, Banca, Orgao, Prova, Disciplina, Assunto, Enunciado, Resposta_Correta, Alternativa_A, Alternativa_B, Alternativa_C, Alternativa_D, Alternativa_E

✅

In [8]:
# ============================================
# CÉLULA 7: TESTE COM 1 QUESTÃO DE PORTUGUÊS
# ============================================

def testar_questao(dataframe, indice=0):
    """Testa análise com 1 questão de Português"""
    
    print("🧪 TESTE - Processando 1 questão de Português")
    print("="*70 + "\n")
    
    try:
        row = dataframe.iloc[indice]
        
        # Montar dicionário da questão
        questao = {
            'id_questao': row.get('ID_Questao', f'Q{indice+1}'),
            'banca': row.get('Banca', 'N/A'),
            'ano': row.get('Ano', 'N/A'),
            'orgao': row.get('Orgao', 'N/A'),
            'prova': row.get('Prova', 'N/A'),
            'disciplina': row.get('Disciplina', 'Português'),
            'assunto': row.get('Assunto', 'N/A'),
            'enunciado': row.get('Enunciado', 'N/A'),
            'alt_a': row.get('Alternativa_A', 'N/A'),
            'alt_b': row.get('Alternativa_B', 'N/A'),
            'alt_c': row.get('Alternativa_C', 'N/A'),
            'alt_d': row.get('Alternativa_D', 'N/A'),
            'alt_e': row.get('Alternativa_E', 'N/A'),
            'gabarito': str(row.get('Resposta_Correta', 'N/A')).strip().upper()
        }
        
        print(f"📋 ID: {questao['id_questao']}")
        print(f"🏛️  Banca: {questao['banca']}")
        print(f"📅 Ano: {questao['ano']} | 🏢 Órgão: {questao['orgao']}")
        print(f"📖 Assunto: {questao['assunto'][:60]}...")
        print(f"✅ Gabarito: {questao['gabarito']}\n")
        
        print("🤖 Analisando com Claude Sonnet 4.5...")
        print("⏳ Aguarde 20-30 segundos (análise profunda)...\n")
        
        # Analisar questão
        analise = analisar_questao_com_ia_otimizada(questao)
        
        # Verificar se houve erro
        if analise.startswith("❌"):
            raise Exception(analise)
        
        # Mostrar preview da análise
        print(f"\n📏 Tamanho da análise: {len(analise)} caracteres")
        print(f"📊 Preview (primeiros 700 caracteres):\n")
        print("="*70)
        print(analise[:700] + "...")
        print("="*70 + "\n")
        
        # Gerar PDF de teste
        print("📄 Gerando PDF de teste...")
        output_path = f'TESTE_PORTUGUES_{questao["id_questao"]}.pdf'
        gerar_pdf_questao(questao, analise, output_path)
        
        print("\n" + "="*70)
        print("✅ TESTE CONCLUÍDO COM SUCESSO!")
        print(f"📁 Arquivo gerado: {output_path}")
        print("="*70)
        print("\n💡 PRÓXIMOS PASSOS:")
        print("   1. Abra o PDF e verifique a qualidade da análise")
        print("   2. Se estiver satisfeito, execute Célula 8 para processar TODAS")
        print("   3. Se quiser testar outra questão, execute:")
        print("      testar_questao(df, indice=5)  # exemplo: questão 6")
        
        return questao, analise
        
    except KeyError as e:
        print(f"❌ ERRO: Coluna não encontrada no dataset!")
        print(f"   Coluna faltando: {e}")
        print("\n🔍 Colunas disponíveis:")
        print(f"   {', '.join(dataframe.columns.tolist())}")
        print("\n💡 SOLUÇÃO:")
        print("   Verifique se os nomes das colunas no Excel correspondem a:")
        print("   - ID_Questao (ou similar)")
        print("   - Banca")
        print("   - Ano")
        print("   - Orgao")
        print("   - Disciplina")
        print("   - Assunto")
        print("   - Enunciado")
        print("   - Alternativa_A, Alternativa_B, etc.")
        print("   - Resposta_Correta")
        return None, None
        
    except Exception as e:
        print(f"❌ ERRO INESPERADO: {str(e)}")
        print("\n🔍 Detalhes técnicos:")
        import traceback
        traceback.print_exc()
        print("\n💡 Se o erro persistir, verifique:")
        print("   1. Conexão com AWS Bedrock")
        print("   2. Permissões do modelo Sonnet 4.5")
        print("   3. Formato dos dados no Excel")
        return None, None

# EXECUTAR TESTE
print("🚀 Iniciando teste com primeira questão do dataset...\n")
questao_teste, analise_teste = testar_questao(df, indice=0)

if questao_teste and analise_teste:
    print("\n" + "🎉"*23)
    print("SISTEMA FUNCIONANDO PERFEITAMENTE!")
    print("🎉"*23)

🚀 Iniciando teste com primeira questão do dataset...

🧪 TESTE - Processando 1 questão de Português

📋 ID: Q3114574
🏛️  Banca: FGV
📅 Ano: 2024 | 🏢 Órgão: TJ-MT
📖 Assunto: Morfologia,...
✅ Gabarito: E

🤖 Analisando com Claude Sonnet 4.5...
⏳ Aguarde 20-30 segundos (análise profunda)...

   📊 Score de qualidade: 100/100
   ✅ Análise aprovada!

📏 Tamanho da análise: 13087 caracteres
📊 Preview (primeiros 700 caracteres):

# 📚 ANÁLISE PEDAGÓGICA COMPLETA - QUESTÃO FGV 2024 (TJ-MT)

---

## 1️⃣ IDENTIFICAÇÃO RÁPIDA
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎯 **Tema Central:** Famílias de Palavras / Campos Semânticos e Etimológicos

📚 **Área:** MORFOLOGIA (Estrutura e Formação de Palavras)

📖 **Subárea:** Análise de Raízes Etimológicas, Campos Lexicais, Derivação Morfológica

🎓 **Nível:** INTERMEDIÁRIO (requer conhecimento de etimologia e raízes latinas)

🎪 **Padrão da Banca FGV:** Questão de exclusão por critério morfológico/etimológico; exige análise profunda de radicais 

In [ ]:
# ============================================
# CÉLULA 8: PROCESSAR TODAS AS QUESTÕES
# ============================================

def processar_questoes(dataframe, pasta_saida='pdfs_portugues'):
    """
    Processa todas as questões: analisa com IA e gera PDFs
    
    Args:
        dataframe: DataFrame com as questões de Português
        pasta_saida: Pasta onde salvar os PDFs
    """
    
    # Criar pasta de saída
    Path(pasta_saida).mkdir(exist_ok=True)
    
    total = len(dataframe)
    sucessos = 0
    erros = 0
    erros_detalhes = []
    
    print("="*70)
    print(f"🚀 PROCESSAMENTO EM LOTE - QUESTÕES DE PORTUGUÊS")
    print("="*70)
    print(f"📊 Total de questões: {total}")
    print(f"📁 Pasta de saída: {pasta_saida}/")
    print(f"🤖 Modelo: Claude Sonnet 4.5")
    print(f"⏱️  Tempo estimado: ~{total * 30} segundos (~{total * 30 / 60:.1f} minutos)")
    print(f"💰 Custo estimado: ~${total * 0.004:.2f} USD")
    print("="*70 + "\n")
    
    confirmacao = input("⚠️  Digite 'SIM' para confirmar e iniciar: ")
    
    if confirmacao.upper() != 'SIM':
        print("❌ Processamento cancelado pelo usuário.")
        return
    
    print("\n🚀 Iniciando processamento em lote...\n")
    print("💡 Dica: Este processo pode demorar. Aguarde até o final!\n")
    
    from datetime import datetime
    inicio = datetime.now()
    
    for index, row in dataframe.iterrows():
        num_questao = index + 1
        
        try:
            print(f"{'='*70}")
            print(f"📌 QUESTÃO {num_questao}/{total} ({num_questao/total*100:.1f}%)")
            print(f"{'='*70}")
            
            # Montar dicionário da questão
            questao = {
                'id_questao': row.get('ID_Questao', f'Q{num_questao}'),
                'banca': row.get('Banca', 'N/A'),
                'ano': row.get('Ano', 'N/A'),
                'orgao': row.get('Orgao', 'N/A'),
                'prova': row.get('Prova', 'N/A'),
                'disciplina': row.get('Disciplina', 'Português'),
                'assunto': row.get('Assunto', 'N/A'),
                'enunciado': row.get('Enunciado', 'N/A'),
                'alt_a': row.get('Alternativa_A', 'N/A'),
                'alt_b': row.get('Alternativa_B', 'N/A'),
                'alt_c': row.get('Alternativa_C', 'N/A'),
                'alt_d': row.get('Alternativa_D', 'N/A'),
                'alt_e': row.get('Alternativa_E', 'N/A'),
                'gabarito': str(row.get('Resposta_Correta', 'N/A')).strip().upper()
            }
            
            print(f"📚 ID: {questao['id_questao']}")
            print(f"🏛️  Banca: {questao['banca']}")
            print(f"📅 Ano: {questao['ano']} | 🏢 Órgão: {questao['orgao']}")
            print(f"📖 Assunto: {questao['assunto'][:50]}...")
            print(f"✅ Gabarito: {questao['gabarito']}\n")
            
            # Analisar com IA (função otimizada)
            print(f"🤖 Analisando com Claude Sonnet 4.5...")
            analise = analisar_questao_com_ia_otimizada(questao)
            
            if analise.startswith("❌"):
                raise Exception(analise)
            
            print(f"✅ Análise recebida ({len(analise)} caracteres)")
            
            # Gerar nome do arquivo
            assunto_limpo = str(questao['assunto']).replace(' ', '_').replace('/', '-')[:40]
            banca_limpa = str(questao['banca']).replace(' ', '_')[:15]
            filename = f"Q{num_questao:03d}_{banca_limpa}_{questao['ano']}_{assunto_limpo}_GAB{questao['gabarito']}.pdf"
            
            # Limpar caracteres especiais do filename
            filename = "".join(c for c in filename if c.isalnum() or c in ('_', '-', '.'))
            output_path = os.path.join(pasta_saida, filename)
            
            # Gerar PDF
            print(f"📄 Gerando PDF...")
            gerar_pdf_questao(questao, analise, output_path)
            
            sucessos += 1
            
            # Mostrar tempo decorrido e estimativa
            tempo_decorrido = (datetime.now() - inicio).total_seconds()
            tempo_medio = tempo_decorrido / num_questao
            tempo_restante = tempo_medio * (total - num_questao)
            
            print(f"✅ Questão {num_questao} concluída!")
            print(f"⏱️  Tempo restante estimado: {tempo_restante/60:.1f} minutos\n")
            
        except Exception as e:
            erros += 1
            erro_msg = f"Q{num_questao} (ID: {questao.get('id_questao', 'N/A')}): {str(e)[:100]}"
            erros_detalhes.append(erro_msg)
            print(f"❌ ERRO: {str(e)[:150]}")
            print(f"   Continuando para próxima questão...\n")
            continue
    
    # Calcular tempo total
    tempo_total = (datetime.now() - inicio).total_seconds()
    
    # Resumo final
    print("\n" + "="*70)
    print("🎉 PROCESSAMENTO CONCLUÍDO!")
    print("="*70)
    print(f"✅ Sucessos: {sucessos}/{total} ({sucessos/total*100:.1f}%)")
    print(f"❌ Erros: {erros}/{total} ({erros/total*100:.1f}%)")
    print(f"⏱️  Tempo total: {tempo_total/60:.1f} minutos")
    print(f"📁 PDFs salvos em: {os.path.abspath(pasta_saida)}/")
    
    if erros > 0:
        print(f"\n⚠️  QUESTÕES COM ERRO:")
        for i, erro in enumerate(erros_detalhes[:10], 1):  # Mostrar só as 10 primeiras
            print(f"   {i}. {erro}")
        if len(erros_detalhes) > 10:
            print(f"   ... e mais {len(erros_detalhes)-10} erros")
    
    print("="*70)
    
    # Gerar arquivo de log
    log_path = os.path.join(pasta_saida, 'log_processamento.txt')
    with open(log_path, 'w', encoding='utf-8') as f:
        f.write(f"PROCESSAMENTO DE QUESTÕES - LÍNGUA PORTUGUESA\n")
        f.write(f"="*50 + "\n")
        f.write(f"Data: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n")
        f.write(f"Modelo: Claude Sonnet 4.5\n")
        f.write(f"Total de questões: {total}\n")
        f.write(f"Sucessos: {sucessos} ({sucessos/total*100:.1f}%)\n")
        f.write(f"Erros: {erros} ({erros/total*100:.1f}%)\n")
        f.write(f"Tempo total: {tempo_total/60:.1f} minutos\n")
        f.write(f"Pasta de saída: {pasta_saida}/\n")
        f.write(f"\n")
        
        if erros > 0:
            f.write(f"DETALHES DOS ERROS:\n")
            f.write(f"="*50 + "\n")
            for i, erro in enumerate(erros_detalhes, 1):
                f.write(f"{i}. {erro}\n")
    
    print(f"📝 Log detalhado salvo em: {log_path}")
    print("\n🎉 Processo finalizado com sucesso!")

print("✅ Função de processamento em lote criada!")
print("\n📋 Para executar, rode:")
print("   processar_questoes(df, pasta_saida='pdfs_portugues')")
print("\n💡 Ou personalize a pasta:")
print("   processar_questoes(df, pasta_saida='meus_pdfs')")
print("\n" + "="*70)
print("⚠️  ATENÇÃO: Não execute ainda se quiser fazer mais testes!")
print("📌 Execute apenas quando estiver pronto para processar TUDO")
print("="*70)

🚀 PROCESSAMENTO EM LOTE - FGV DIREITO ADMINISTRATIVO
📊 Total de questões: 220
📁 Pasta de saída: pdfs_fgv_direito_adm/
⏱️  Tempo estimado: ~5500 segundos (~91.7 minutos)
💰 Custo estimado: ~$0.66 USD



KeyboardInterrupt: Interrupted by user

In [ ]:
# ============================================
# CÉLULA 9: ESTATÍSTICAS DETALHADAS
# ============================================

import glob

def gerar_relatorio_completo(dataframe, pasta='pdfs_fgv_direito_adm'):
    """Gera relatório completo do processamento"""
    
    print("="*70)
    print("📊 RELATÓRIO COMPLETO DO PROCESSAMENTO")
    print("="*70 + "\n")
    
    # Contar PDFs gerados
    pdfs = glob.glob(f"{pasta}/*.pdf")
    total_pdfs = len(pdfs)
    
    print(f"📄 PDFs Gerados: {total_pdfs}/{len(dataframe)}")
    
    # Tamanho total
    if pdfs:
        tamanho_total = sum(os.path.getsize(pdf) for pdf in pdfs)
        tamanho_mb = tamanho_total / (1024 * 1024)
        print(f"💾 Tamanho total: {tamanho_mb:.2f} MB")
        print(f"📏 Tamanho médio: {tamanho_mb/total_pdfs:.2f} MB por PDF")
    
    # Análise por ano
    print(f"\n📅 QUESTÕES POR ANO:")
    for ano, count in sorted(dataframe['Ano'].value_counts().items(), reverse=True):
        porcentagem = (count / len(dataframe)) * 100
        barra = "█" * int(porcentagem / 2)
        print(f"   {ano}: {count:3d} questões ({porcentagem:5.1f}%) {barra}")
    
    # Análise por órgão (top 10)
    print(f"\n🏢 TOP 10 ÓRGÃOS:")
    for orgao, count in dataframe['Orgao'].value_counts().head(10).items():
        porcentagem = (count / len(dataframe)) * 100
        print(f"   {orgao:20s}: {count:3d} ({porcentagem:5.1f}%)")
    
    # Análise por assunto (top 10)
    print(f"\n📖 TOP 10 ASSUNTOS:")
    for assunto, count in dataframe['Assunto'].value_counts().head(10).items():
        assunto_curto = assunto[:50] + "..." if len(assunto) > 50 else assunto
        porcentagem = (count / len(dataframe)) * 100
        print(f"   {count:3d} ({porcentagem:4.1f}%): {assunto_curto}")
    
    # Distribuição de gabaritos
    print(f"\n✅ DISTRIBUIÇÃO DE GABARITOS:")
    gabaritos = dataframe['Resposta_Correta'].value_counts().sort_index()
    for gab, count in gabaritos.items():
        porcentagem = (count / len(dataframe)) * 100
        barra = "▓" * int(porcentagem)
        print(f"   Alternativa {gab}: {count:3d} ({porcentagem:5.1f}%) {barra}")
    
    # Teste qui-quadrado para verificar se distribuição é uniforme
    esperado = len(dataframe) / 5
    chi_quadrado = sum((count - esperado)**2 / esperado for count in gabaritos.values())
    print(f"\n📈 Análise estatística da distribuição:")
    print(f"   Valor esperado por alternativa: {esperado:.1f}")
    print(f"   Chi-quadrado: {chi_quadrado:.2f}")
    if chi_quadrado < 9.49:  # Valor crítico para 4 graus de liberdade, 95% confiança
        print(f"   ✅ Distribuição aproximadamente uniforme")
    else:
        print(f"   ⚠️  Distribuição não uniforme")
    
    print("\n" + "="*70)
    print(f"📁 Arquivos em: {os.path.abspath(pasta)}/")
    print("="*70)

# Gerar relatório
gerar_relatorio_completo(df, pasta='pdfs_fgv_direito_adm')

In [9]:
# ============================================
# CÉLULA DEBUG: DIAGNOSTICAR LENTIDÃO
# ============================================

import time

def testar_velocidade(questao_dict):
    """Testa velocidade e identifica gargalos"""
    
    print("🔍 DIAGNÓSTICO DE PERFORMANCE\n")
    print("="*70)
    
    # Teste 1: API Bedrock está respondendo?
    print("\n1️⃣ Testando conexão com AWS Bedrock...")
    inicio = time.time()
    
    try:
        test_config = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 50,  # Mínimo para teste rápido
            "temperature": 0.3,
            "messages": [{"role": "user", "content": "Responda apenas 'OK'"}]
        }
        
        response = bedrock_client.invoke_model(
            body=json.dumps(test_config),
            modelId=MODEL_ID,
            accept="application/json",
            contentType="application/json"
        )
        
        tempo_api = time.time() - inicio
        print(f"   ✅ API respondeu em {tempo_api:.2f} segundos")
        
        if tempo_api > 5:
            print("   ⚠️  PROBLEMA: API está lenta (deveria ser < 2s)")
            
    except Exception as e:
        print(f"   ❌ ERRO na API: {str(e)}")
        return
    
    # Teste 2: Análise completa
    print("\n2️⃣ Testando análise completa...")
    inicio = time.time()
    
    try:
        analise = analisar_questao_com_ia_otimizada(questao_dict)
        tempo_total = time.time() - inicio
        
        print(f"   ✅ Análise concluída em {tempo_total:.2f} segundos")
        print(f"   📏 Tamanho: {len(analise)} caracteres")
        
        if tempo_total > 60:
            print("   🚨 PROBLEMA GRAVE: Tempo > 1 minuto!")
        elif tempo_total > 30:
            print("   ⚠️  Um pouco lento, mas aceitável")
        else:
            print("   ✅ Tempo normal!")
            
    except Exception as e:
        print(f"   ❌ ERRO: {str(e)}")
        return
    
    # Teste 3: Verificar cache
    print("\n3️⃣ Testando cache...")
    inicio = time.time()
    
    analise2 = analisar_questao_com_ia_otimizada(questao_dict)
    tempo_cache = time.time() - inicio
    
    print(f"   ⏱️  Segunda chamada: {tempo_cache:.2f} segundos")
    
    if tempo_cache < 1:
        print("   ✅ Cache funcionando perfeitamente!")
    else:
        print("   ⚠️  Cache pode não estar funcionando")
    
    print("\n" + "="*70)
    print("📊 RESUMO:")
    print(f"   API básica: {tempo_api:.2f}s")
    print(f"   Análise completa: {tempo_total:.2f}s")
    print(f"   Com cache: {tempo_cache:.2f}s")
    print("="*70)
    
    # Diagnóstico
    print("\n💡 DIAGNÓSTICO:")
    if tempo_api > 5:
        print("   🔴 Problema: Conexão AWS/Bedrock lenta")
        print("      Solução: Verifique região, rede, credenciais")
    elif tempo_total > 90:
        print("   🔴 Problema: max_tokens muito alto ou retry infinito")
        print("      Solução: Reduza max_tokens para 3000")
    elif tempo_total > 45:
        print("   🟡 Problema: Score baixo causando retries")
        print("      Solução: Reduza score mínimo para 50")
    else:
        print("   🟢 Tudo normal! Apenas o modelo Sonnet é mais lento que Haiku")

# EXECUTAR DIAGNÓSTICO
if 'questao_teste' in globals() and questao_teste:
    testar_velocidade(questao_teste)
else:
    print("❌ Execute primeiro a Célula 7 para ter uma questão de teste")

❌ Execute primeiro a Célula 7 para ter uma questão de teste
